# Asset Portfolio Management using Deep Reinforcement Learning
----

## 3.0 Stock Selection
---
Auto Encoders are employed to select the less volatile stocks by choosing stocks with less reconstruction error. These are the stocks that are included in the portfolio

### 3.1 Import Relevant Libraries

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [28]:
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import regularizers, optimizers

from sklearn import preprocessing

### 3.2 Load the Data

In [29]:
# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

In [30]:
df = prices_data.copy()

In [31]:
df = df.reset_index(drop=True).set_index(['date'])

In [32]:
df.head()

,WBA,TRV,MMM,RTX,DD,KO,V,XOM,CAT,PG,...,JNJ,CSCO,PFE,IBM,MRK,INTC,MCD,BA,NKE,GS
date,,,,,,,,,,,,,,,,,,,,,
2008-03-19,27.162476,33.676067,55.994167,31.819752,34.389133,20.135204,12.927960,54.472538,51.048386,45.970528,...,44.239494,18.299444,11.740806,80.073196,26.932304,14.305408,36.734047,54.094521,13.067656,138.832825
2008-03-20,27.461124,34.677429,54.944336,31.935442,34.455250,20.497879,14.724145,54.840305,51.110714,47.028114,...,44.580441,18.523792,11.729413,81.024979,27.164101,14.753081,37.254330,55.088783,14.217390,149.790054
2008-03-24,28.827457,34.604851,55.409344,32.550919,35.569756,20.544893,13.667027,55.453220,52.640846,47.434856,...,44.246326,19.174410,11.740806,81.524841,27.721666,15.010838,38.178493,56.186138,14.599930,149.164658
2008-03-25,28.379486,34.655636,55.648911,32.467625,35.758652,20.622128,14.472454,54.969315,53.063187,47.143330,...,44.014500,19.256668,11.837650,80.778450,27.984785,15.105799,38.472866,55.898911,14.449876,149.790054
2008-03-26,28.342155,34.503265,55.423466,32.213104,35.711433,20.534811,14.634912,55.653210,53.561695,47.211140,...,44.116768,18.531269,11.786383,80.052643,28.003590,14.827696,38.062122,56.193504,14.006041,146.329437


### 3.3 Define Functions

In [33]:
def defineAutoencoder(num_stock, encoding_dim = 5, verbose=0):
    
    """
    Function for fitting an Autoencoder
    """

    # connect all layers
    input = Input(shape=(num_stock,))

    encoded = Dense(encoding_dim, kernel_regularizer=regularizers.l2(0.00001),name ='Encoder_Input')(input)

    decoded = Dense(num_stock, kernel_regularizer=regularizers.l2(0.00001), name ='Decoder_Input')(encoded)
    decoded = Activation("linear", name='Decoder_Activation_function')(decoded)

    # construct and compile AE model
    autoencoder = Model(inputs=input, outputs=decoded)
    adam = optimizers.Adam(lr=0.0005)
    autoencoder.compile(optimizer=adam, loss='mean_squared_error')
    if verbose!= 0:
        autoencoder.summary()

    return autoencoder

In [34]:
def getReconstructionErrorsDF(df_pct_change, reconstructed_data):
    
    """
    Function for calculating the reconstruction Errors
    """
    array = []
    stocks_ranked = []
    num_columns = reconstructed_data.shape[1]
    for i in range(0, num_columns):
        diff = np.linalg.norm((df_pct_change.iloc[:, i] - reconstructed_data[:, i]))  # 2 norm difference
        array.append(float(diff))

    ranking = np.array(array).argsort()
    r = 1
    for stock_index in ranking:
        stocks_ranked.append([ r
                              ,stock_index
                              ,df_pct_change.iloc[:, stock_index].name
                              ,array[stock_index]
                              ])
        r = r + 1

    columns = ['ranking','stock_index', 'stock_name' ,'recreation_error']
    df = pd.DataFrame(stocks_ranked, columns=columns)
    df = df.set_index('stock_name')
    return df

### 3.4 Get the Percentage Change of the Close Prices

In [35]:
col_names = df.columns.to_list()

In [36]:
print(col_names)

['WBA', 'TRV', 'MMM', 'RTX', 'DD', 'KO', 'V', 'XOM', 'CAT', 'PG', 'AXP', 'WMT', 'MSFT', 'VZ', 'JPM', 'UNH', 'DIS', 'HD', 'AAPL', 'CVX', 'JNJ', 'CSCO', 'PFE', 'IBM', 'MRK', 'INTC', 'MCD', 'BA', 'NKE', 'GS']


In [37]:
df_pct_change = df.pct_change(1).astype(float)
df_pct_change = df_pct_change.replace([np.inf, -np.inf], np.nan)
df_pct_change = df_pct_change.fillna(method='ffill')

# the percentage change function will make the first two rows equal to nan
df_pct_change = df_pct_change.tail(len(df_pct_change) - 2)

In [38]:
df_pct_change.shape

(3219, 30)

In [39]:
# remove columns where there is no change over a longer time period
df_pct_change = df_pct_change[df_pct_change.columns[((df_pct_change == 0).mean() <= 0.05)]]

In [40]:
df_pct_change.head()

,WBA,TRV,MMM,RTX,DD,KO,V,XOM,CAT,PG,...,JNJ,CSCO,PFE,IBM,MRK,INTC,MCD,BA,NKE,GS
date,,,,,,,,,,,,,,,,,,,,,
2008-03-24,0.049755,-0.002093,0.008463,0.019273,0.032346,0.002294,-0.071795,0.011176,0.029938,0.008649,...,-0.007495,0.035123,0.000971,0.006169,0.020526,0.017471,0.024807,0.019920,0.026906,-0.004175
2008-03-25,-0.015540,0.001468,0.004324,-0.002559,0.005311,0.003759,0.058932,-0.008726,0.008023,-0.006146,...,-0.005239,0.004290,0.008249,-0.009155,0.009491,0.006326,0.007710,-0.005112,-0.010278,0.004193
2008-03-26,-0.001315,-0.004397,-0.004051,-0.007839,-0.001320,-0.004234,0.011225,0.012441,0.009395,0.001438,...,0.002324,-0.037670,-0.004331,-0.008985,0.000672,-0.018410,-0.010676,0.005270,-0.030716,-0.023103
2008-03-27,-0.018440,-0.001893,-0.010171,-0.002874,-0.013640,0.002617,-0.016417,-0.000695,-0.004265,-0.003303,...,-0.001236,-0.024213,-0.006283,-0.011889,0.001789,-0.035224,-0.002698,-0.027261,-0.004376,-0.041828
2008-03-28,0.004562,-0.000843,0.001156,-0.012390,-0.005965,-0.006035,-0.002384,-0.011369,0.000778,0.000432,...,-0.006809,-0.004136,-0.002918,-0.008224,-0.006029,-0.014225,0.000541,-0.010105,-0.001819,-0.021946


### 3.5 Construct the Autoencoder

In [68]:
# define the input parameters
hidden_layers = 5
batch_size = 500
epochs = 500
stock_selection_number = 20
num_stock = df_pct_change.shape[1]
verbose = 1

In [69]:
print('-' * 20 + 'Step 1 : Returns vs. recreation error (recreation_error)')
print('-' * 25 + 'Transform dataset with MinMax Scaler')

--------------------Step 1 : Returns vs. recreation error (recreation_error)
-------------------------Transform dataset with MinMax Scaler


In [70]:
# Normalize the data
df_scaler = preprocessing.MinMaxScaler()
df_pct_change_normalised = df_scaler.fit_transform(df_pct_change)

In [71]:
# define autoencoder
print('-' * 25 + 'Define autoencoder model')
num_stock = len(df_pct_change.columns)
autoencoder = defineAutoencoder(num_stock=num_stock, encoding_dim=hidden_layers, verbose=verbose)
#plot_model(autoencoder, to_file='img/model_autoencoder_1.png', show_shapes=True,
#           show_layer_names=True)

-------------------------Define autoencoder model
Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
Encoder_Input (Dense)        (None, 5)                 155       
_________________________________________________________________
Decoder_Input (Dense)        (None, 30)                180       
_________________________________________________________________
Decoder_Activation_function  (None, 30)                0         
Total params: 335
Trainable params: 335
Non-trainable params: 0
_________________________________________________________________


In [72]:
# train autoencoder
print('-' * 25 + 'Train autoencoder model')
autoencoder.fit(df_pct_change_normalised, df_pct_change_normalised, shuffle=False, epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)

-------------------------Train autoencoder model
Epoch 1/500
7/7 [==============================] - 0s 3ms/step - loss: 0.2792
Epoch 2/500
7/7 [==============================] - 0s 2ms/step - loss: 0.2576
Epoch 3/500
7/7 [==============================] - 0s 2ms/step - loss: 0.2391
Epoch 4/500
7/7 [==============================] - 0s 2ms/step - loss: 0.2234
Epoch 5/500
7/7 [==============================] - 0s 2ms/step - loss: 0.2096
Epoch 6/500
7/7 [==============================] - 0s 2ms/step - loss: 0.1974
Epoch 7/500
7/7 [==============================] - 0s 3ms/step - loss: 0.1863
Epoch 8/500
7/7 [==============================] - 0s 2ms/step - loss: 0.1759
Epoch 9/500
7/7 [==============================] - 0s 5ms/step - loss: 0.1662
Epoch 10/500
7/7 [==============================] - 0s 2ms/step - loss: 0.1569
Epoch 11/500
7/7 [==============================] - 0s 2ms/step - loss: 0.1479
Epoch 12/500
7/7 [==============================] - 0s 3ms/step - loss: 0.1393
Epoch 13/500

7/7 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 104/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 105/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 106/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 107/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 108/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 109/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 110/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 111/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 112/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 113/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 114/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 115/500
7/7 [==============================] - 0

7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 206/500
7/7 [==============================] - 0s 1ms/step - loss: 0.0022
Epoch 207/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 208/500
7/7 [==============================] - 0s 1ms/step - loss: 0.0022
Epoch 209/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 210/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 211/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 212/500
7/7 [==============================] - 0s 1ms/step - loss: 0.0022
Epoch 213/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 214/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 215/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 216/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 217/500
7/7 [==============================] - 0

7/7 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 308/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 309/500
7/7 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 310/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 311/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 312/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 313/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 314/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 315/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 316/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 317/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 318/500
7/7 [==============================] - ETA: 0s - loss: 0.003 - 0s 2ms/step - loss: 0.0020
Epoch 319/500
7/7 [===========

7/7 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 409/500
7/7 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 410/500
7/7 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 411/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 412/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 413/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 414/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 415/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 416/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 417/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 418/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 419/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 420/500
7/7 [==============================] - 0

In [73]:
# predict autoencoder
print('-' * 25 + 'Predict autoencoder model')
reconstruct = autoencoder.predict(df_pct_change_normalised)

-------------------------Predict autoencoder model


In [74]:
# Inverse transform dataset with MinMax Scaler
print('-' * 25 + 'Inverse transform dataset with MinMax Scaler')
reconstruct_real = df_scaler.inverse_transform(reconstruct)
df_reconstruct_real = pd.DataFrame(data=reconstruct_real, columns=df_pct_change.columns)

-------------------------Inverse transform dataset with MinMax Scaler


In [75]:
print('-' * 25 + 'Calculate L2 norm as reconstruction loss metric')
df_recreation_error = getReconstructionErrorsDF(df_pct_change=df_pct_change,
                                                reconstructed_data=reconstruct_real)

-------------------------Calculate L2 norm as reconstruction loss metric


In [76]:
df_recreation_error

,ranking,stock_index,recreation_error
stock_name,,,
JNJ,1,20,0.451467
PG,2,9,0.472419
MMM,3,2,0.541604
KO,4,5,0.550200
IBM,5,23,0.565008
VZ,6,13,0.569757
MCD,7,26,0.595712
PFE,8,22,0.601264
RTX,9,3,0.609634


In [77]:
filtered_stocks = df_recreation_error.head(stock_selection_number).index

In [78]:
filtered_stocks

Index(['JNJ', 'PG', 'MMM', 'KO', 'IBM', 'VZ', 'MCD', 'PFE', 'RTX', 'WMT',
       'MRK', 'V', 'DIS', 'MSFT', 'XOM', 'HD', 'TRV', 'INTC', 'AXP', 'NKE'],
      dtype='object', name='stock_name')

In [79]:
# store the list of selected stocks
%store filtered_stocks

Stored 'filtered_stocks' (Index)
